## Imports

In [ ]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

import h5py
import os
import numpy as np
from typing import Dict, List, Optional, Tuple

from dataclasses import dataclass
from tqdm.auto import tqdm
from scipy.signal import savgol_filter
from scipy.interpolate import interp2d
from functools import lru_cache


import plotly.graph_objects as go
import plotly.colors as pc
import matplotlib.pyplot as plt


import sys

sys.path.append(r"C:\Users\atully\Code\GitHub\ARPES Code\arpes-code-python")
from arpes_functions import (
    fitting_functions,
    analysis_functions,
    plotting_functions,
    HDF5_loader,
    misc_functions,
    filter_functions,
    tr_functions,
    loading_functions,
    kw_data_loader,
    cnn,
    polygons,
)

colors = pc.qualitative.D3
angstrom = "\u212B"
Theta = "\u0398"
phi = "\u03C6"

# Convert Data

In [ ]:
# ## Convert K corrected .ibw data to .h5 ##

# ddir = r"E:\atully\arpes_data\2023_May\May13\C60\FS"

# # STEP 1 ##
# # Convert ibw to hdf5
# fn = "FS0and2_avg_gkw.ibw"
# HDF5_loader.ibw_to_hdf5(ddir, fn, export=True)

# # Check conversion worked
# data, kx, ky, energy = HDF5_loader.load_hdf5(
#     ddir, "FS0and2_avg_gkw.h5"
# )  # load data from hdf5
# # data.shape, kx.shape, ky.shape, energy.shape

# Set Up Parameters

In [ ]:
EF_400 = 1.91  # in kinetic energy, slit 400
EF_700 = 1.94  # in kinetic energy, slit 700

homo = -2.05

homo_400 = homo + EF_400

# Load Data

In [ ]:
## Load Data ##

# ddir = r"E:\atully\arpes_data\2023_February\6eV\FS"
# files = ["FS_4and10_gkw.h5"]  # CT1, neg delay
# files = ["FS1_FS3_Ali_avg_gkw.h5"]  # CT1
# files = ["FS8_avg_gkw.h5"]  # left side of CT1
# files = ["FS2_avg_gkw.h5"]  # CT2

ddir = r"E:\atully\arpes_data\2023_June\C60\ARPES\FS"
files = ["FS8&9&10_Ali_avg.h5"]  # CT2

# ddir = r"E:\atully\arpes_data\2023_May\May13\C60\FS"
# files = ["FS4_Ali_avg.h5"]  # CT2
# files = ["FS0&FS2_Ali_avg.h5"]  # CT1 -- high stats
# files = ["FS5_Ali_avg.h5"]  # CT1 -- centered on G
# files = ["FS0and2_avg_gkw.h5"]  # CT1 -- high stats, k-corrected

# This works, but makes dataclass with theta and phi_or_time instead of kx and ky
ARPES_DATA: Dict[str, tr_functions.ArpesData] = {}
ARPES_ATTRS: Dict[str, tr_functions.ArpesAttrs] = {}
for file in tqdm(files):
    data, kx, ky, energy = loading_functions.load_hdf5(ddir, file)
    ARPES_DATA[file] = tr_functions.ArpesData(
        data=data, theta=kx, phi_or_time=ky, energy=energy
    )
    ARPES_ATTRS[file] = tr_functions.load_attrs_hdf5(ddir, file)

ad = ARPES_DATA[files[0]]

ad.energy = ad.energy - homo_400

In [ ]:
ad_ct2_june = ad
# ad_ct2_may = ad
# ad_ct1_may = ad
# ad_ct1_may_centered = ad

# ad_ct1_may_kw = ad
# ad_ct1_feb_kw = ad
# ad_ct2_feb_kw = ad

# Analysis

In [ ]:
slice_val = 2.0
slice_val = 2.1
slice_val = 2.2
# slice_val = 2.3
# slice_val = 2.4
# slice_val = 2.5
# slice_val = 2.6
# slice_val = 2.7
# slice_val = 2.75
# slice_val = 2.8
# slice_val = 2.9
slice_val = 3.0
# slice_val = 3.1


slice_dim = "y"
int_range = 0.1

## raw ##
xlim = (-19, 16)
ylim = None

## k-corrected ##
# ylim = (-0.52, 0.17)  # Feb CT1
# xlim = (-0.12, 0.45)  # Feb CT1

# ylim = (-0.57, 0.18)  # Feb CT2
# xlim = (-0.13, 0.47)  # Feb CT2

# ylim = (-0.4, 0.07)  # May
# xlim = (-0.09, 0.33)  # May
# ylim = None
# xlim = None

x_bin = 2
y_bin = 2

In [ ]:
title = f"E - E<sub>HOMO</sub> = {slice_val} eV"

yaxis_title = f"{phi}"
xaxis_title = f"{Theta}"

# xaxis_title = f"k<sub>x</sub> ({angstrom}<sup>-1</sup>)"
# yaxis_title = f"k<sub>y</sub> ({angstrom}<sup>-1</sup>)"

In [ ]:
## Get Slices ##

ad_plot = ad_ct2_june
# ad_plot = ad_ct1_may
# ad_plot = ad_ct1_may_centered
# ad_plot = ad_ct2_may

# ad_plot = ad_ct1_may_kw
# ad_plot = ad_ct1_feb_kw

x, y, d = tr_functions.slice_datacube(
    ad_dataclass=ad_plot,
    slice_dim=slice_dim,
    slice_val=slice_val,
    int_range=int_range,
    xlim=xlim,
    ylim=ylim,
    x_bin=x_bin,
    y_bin=y_bin,
    norm_data=True,
    plot_data=False,
)

In [ ]:
## Plot Data ##
x_plot, y_plot, d_plot = x, y, d

fig = tr_functions.thesis_fig(
    title=title,
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    equiv_axes=False,
    height=500,
    width=500,
)

fig.add_trace(
    go.Heatmap(
        x=x_plot,
        y=y_plot,
        z=analysis_functions.norm_data(d_plot),
        coloraxis="coloraxis",
    )
)

fig.update_coloraxes(cmin=0, cmax=1)

fig.show()

# Reconstruct Full BZ

In [ ]:
## Rotate Data ##

x, y, z = x_plot, y_plot, d_plot
coords = tr_functions.x_y_to_coords(x, y)

# rotated_coords = tr_functions.rotate_2d_array(coords, 120, (0, 0))
# rotated_coords_2 = tr_functions.rotate_2d_array(coords, 240, (0, 0))

rotated_coords = tr_functions.rotate_2d_array(coords, 120, (-12.7, 1))  # June ct2
rotated_coords_2 = tr_functions.rotate_2d_array(coords, 240, (-12.7, 1))

nx, ny, nd = tr_functions.interpolate(rotated_coords, z)
nx_2, ny_2, nd_2 = tr_functions.interpolate(rotated_coords_2, z)

In [ ]:
## Plot raw data and rotations on same figure ##

fig = tr_functions.thesis_fig(
    title=title,
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    equiv_axes=True,
    gridlines=False,
    height=600,
    width=600,
)

fig.add_trace(
    go.Heatmap(
        x=nx_2,
        y=ny_2,
        z=analysis_functions.norm_data(nd_2),
        coloraxis="coloraxis",
        # opacity=0.85,
    )
)

fig.add_trace(
    go.Heatmap(
        x=nx,
        y=ny,
        z=analysis_functions.norm_data(nd),
        coloraxis="coloraxis",
        # opacity=0.85,
    )
)

fig.add_trace(
    go.Heatmap(
        x=x,
        y=y,
        z=analysis_functions.norm_data(z),
        coloraxis="coloraxis",
        # opacity=0.85,
    )
)

# hexagon = polygons.gen_polygon(6, 25, rotation=30, translation=(-12.7, 1))
# fig = polygons.plot_polygon(
#     hexagon, color="firebrick", fig=fig, show=False, dash=True, dash_width=3
# )
hexagon = polygons.gen_polygon(6, 0.42, rotation=30)
fig = polygons.plot_polygon(
    hexagon, color="yellow", fig=fig, show=False, dash=True, dash_width=3
)

fig.update_coloraxes(
    colorscale="ice",
    reversescale=True,
    showscale=True,
    cmin=0,
    cmax=None,
    # cmax=0.6,
)
fig.update_yaxes(scaleanchor="x", scaleratio=1)
fig.show()